In [9]:
import pandas as pd
import math
import numpy as np

In [70]:
df = pd.read_csv('lsmt_data.csv')
df

,Unnamed: 0,timestamp,userID,expiring,sessionID_idx,url_idx
0,0,1676373343936,user-1003,1676373495816,0,0
1,1,1676373344344,user-1005,1676373476316,1,1
2,2,1676373344456,user-1003,1676373495816,0,2
3,3,1676373344912,user-65,1676373604676,2,3
4,4,1676373345020,user-1005,1676373476316,1,4
...,...,...,...,...,...,...
263206,263206,1676444990852,user-629,1676445248826,9002,188
263207,263207,1676444991104,user-629,1676445248826,9002,2921
263208,263208,1676444991342,user-629,1676445248826,9002,2922
263209,263209,1676444991375,user-1005,1676445040536,8965,1155


In [71]:
df_spammers_scrapers = df.loc[(df['userID'] == ' user-1003') | (df['userID'] == ' user-1004') | (df['userID'] == ' user-1005')]
#df_spammers_scrapers

In [72]:
df_normal_users = pd.concat([df, df_spammers_scrapers]).drop_duplicates(keep = False)
#df_normal_users

In [73]:
# Add 30000 normal user instances into the abnormal ones for testing
df_test = pd.concat([df_spammers_scrapers, df_normal_users.iloc[:30000]], ignore_index=True)

# Remove the corresponding rows from the normal ones
df_train = df_normal_users.drop(df_normal_users.index[:30000])

In [74]:
# sort the training data (only normal users) by userID. Then drop column userID and save into csv
df_train = df_train.sort_values(by=['userID'])
df_train = df_train.drop('userID', axis = 1)
df_train.to_csv('train_sorted.csv', index=False)

TODO: 
    Sort the test (normal but mostly abnormal ones) by userID. Delete the column userID, but save into list later
    in order to identify which users are classified as anomalies

Dont need this below for now

In [13]:
#This function will only be used for training data
def split_user_df(dataframe, user):

    #Dataframe containing all requests made by chosen user
    user_data = dataframe.loc[dataframe['userID'] == user]
    number_of_reqs = user_data.shape[0]
   
    partitions = number_of_reqs/50
    partitions = math.ceil(partitions)

    #Splits the data frame into smaller chunks of ~50 requests
    return  np.array_split(user_data, partitions)

In [14]:
#Returns a list of users
def extract_users(dataframe):
    return list(dataframe['userID'].unique())

In [29]:
users = extract_users(df_normal_users)

In [31]:
df_chunks = split_user_df(df_normal_users, users[0])


[       Unnamed: 0      timestamp    userID       expiring  sessionID_idx  \
 3               3  1676373344912   user-65  1676373604676              2   
 21             21  1676373347593   user-65  1676373604676              2   
 24             24  1676373347955   user-65  1676373604676              2   
 27             27  1676373348304   user-65  1676373604676              2   
 64             64  1676373356813   user-65  1676373604676              2   
 65             65  1676373356846   user-65  1676373604676              2   
 67             67  1676373357342   user-65  1676373604676              2   
 68             68  1676373357828   user-65  1676373604676              2   
 69             69  1676373357846   user-65  1676373604676              2   
 70             70  1676373357866   user-65  1676373604676              2   
 71             71  1676373357886   user-65  1676373604676              2   
 72             72  1676373357906   user-65  1676373604676              2   

In [25]:
df_chunks[0].to_csv('datachunk_normal_lstm.csv', index=False)